# Homework 2: Discovery of Frequent Itemsets and Association Rules

This homework has been done by Group 9, the students are Anna Kovács and Alex Orlandi.

The task is to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions, and develop and implement an algorithm for generating association rules between frequent itemsets discovered using the A-Priori algorithm. We wrote the code on this Google Colab.

## Initialization

In this section we set up the environment for performing distributed data processing with PySpark. We set up Spark to run locally within Google Colab.

In [1]:
!pip install findspark

In [2]:
from pyspark import SparkContext, SparkConf # Spark's core classes

import findspark # To set up the environment for pyspark

from google.colab import files
from itertools import combinations
import time

In [3]:
findspark.init() # findspark's initialization

conf = SparkConf().setAppName("Homework_2_frequent_itemsets").setMaster("local[*]") # Local mode
sc = SparkContext(conf=conf) # Entry point for Spark's functionalities

## Dataset

We used the dataset called "transactions.txt", we created it in order to easily visualize the correct functioning of our code. It's a sales transactions dataset, where the number of rows is 13. The dataset is available here:
https://drive.google.com/file/d/1UCqxACBmXkd-VRf0v55MMOGn6KBbV1EW/view?usp=sharing

In [4]:
uploaded = files.upload()
file_name = "transactions.txt"
transactions = sc.textFile("file:" + "/content/" + file_name).map(lambda line: line.strip().split(" "))

Saving transactions.txt to transactions (2).txt


In [5]:
print("First 10 rows:\n")
first_10_lines = transactions.take(10)
for i, line in enumerate(first_10_lines, 1):
    print(f"{i}: {line}")

total_lines = transactions.count()
print(f"\nTotal number of lines: {total_lines}")

First 10 rows:

1: ['1', '2', '3', '4']
2: ['1', '3']
3: ['1', '4', '5', '6']
4: ['2', '3', '4', '5']
5: ['2', '3', '4']
6: ['5', '2']
7: ['5', '1']
8: ['1', '4', '5']
9: ['1']
10: ['1', '2']

Total number of lines: 13


## Parameters

Here we set up the parameters that we will use from now on. These include s, which is the support threshold (the minimum fraction of transactions in which an itemset must appear to be considered frequent). We used 0.2, meaning that an itemset must appear in at least 20% of the transactions, because we thought it is a reasonable percentage, not too low (almost all itemsets included) and not too high (no itemsets included). Then we defined number_of_transactions (the total amount of transactions), frequency_threshold (the minimum support count for an itemset to be considered frequent), and the minimum confidence threshold, that we will use later for creating the associations rules. We chose 0.6 because we thought it balances the trade-off between finding meaningful rules and not being too strict.

In [6]:
s = 0.2 # Support threshold
number_of_transactions = transactions.count() # Total count of transactions
frequency_threshold = s * number_of_transactions # Minimum support count for an itemset to be considered frequent
c = 0.6 # Minimum confidence threshold (used for creating the associations rules)

print(frequency_threshold)

2.6


## A-Priori Algorithm

Core idea of the A-Priori Algorithm: if an itemset is frequent, all its subsets must also be frequent. On the other hand, if an itemset is infrequent, all its supersets are guaranteed to be infrequent.

The algorithm consists of 5 steps:

1) Find frequent 1-itemsets

2) Generate candidate k-itemsets

3) Prune candidates

4) Count support for candidates

5) Iterate

In [7]:
def generate_candidates(previous_freq_itemsets, freq_1_itemsets, k):
    """

    In this function we generate k-itemset candidates by pairing
    (k-1)-itemsets with 1-itemsets. For each (k-1)-itemset, we try
    to add each 1-itemset, creating a new candidate if it has exaclty
    k items.

    """

    print(f"Previous freq itemsets: {previous_freq_itemsets}")
    print(f"\n1 freq itemsets: {freq_1_itemsets}")
    print(f"\nGenerate candidates for k={k}" )

    candidates = set()

    for itemset in previous_freq_itemsets:
        for item in freq_1_itemsets:
            # Create a new candidate by adding the 1-itemset to the (k-1)-itemset
            candidate = itemset | item

            # Only add if the resulting candidate has exactly k items
            if len(candidate) == k:
                candidates.add(candidate)

    print(candidates)

    return candidates


def prune_candidates(candidates, previous_freq_itemsets):
    """

    Here we filter out candidates that contain any infrequent subset.
    We iterate through each candidate itemset and check if every subset
    of size k-1 is infrequent. If any subset is missing, the candidate
    is pruned.

    """

    pruned_candidates = set()

    for candidate in candidates:
        is_valid = True

        # Generate all possible (k-1)-itemsets by removing one item at a time
        for item in candidate:
            subset = candidate - frozenset([item])

            # Check if the subset is in the frequent (k-1)-itemsets
            if subset not in previous_freq_itemsets:
                is_valid = False
                break

        # If all (k-1)-subsets are frequent, add candidate to the pruned set
        if is_valid:
            pruned_candidates.add(candidate)

    print(f"\nPruned candidates: {pruned_candidates}")

    return pruned_candidates

In [8]:
start_time_frequent_itemsets = time.time()

frequent_itemsets = dict() # Frequent 1-itemsets
s_count = dict() # Support counts for frequent itemsets

# Flatten items in each transaction into pairs of (item, 1) and
# sum the counts, giving the support count of each item
item_appear = transactions.flatMap(lambda items: [(item, 1) for item in items]).reduceByKey(lambda x, y: x+y)

# Filter itemsets to keep only those with support >= frequency_threshold
frequent_1_itemsets_tuples = item_appear.filter(lambda item: item[1] >= frequency_threshold).map(lambda item: (frozenset([item[0]]), item[1])).collect()

# frequent_1_itemsets stores frequent 1-itemsets with their counts
frequent_1_itemsets = set()

for itemset, count in frequent_1_itemsets_tuples:
    frequent_itemsets.setdefault(1, set()).add(itemset)
    s_count[itemset] = count
    frequent_1_itemsets.update(itemset)

print(len(frequent_1_itemsets))
print(s_count)

# Convert individual items in freq_1_itemsets to frozensets (immutable sets)
frequent_1_itemsets = set(frozenset([item]) for item in frequent_1_itemsets)

print(f"\nfreq_1_itemsets: {frequent_1_itemsets}")

5
{frozenset({'1'}): 10, frozenset({'4'}): 5, frozenset({'2'}): 8, frozenset({'3'}): 4, frozenset({'5'}): 5}

freq_1_itemsets: {frozenset({'5'}), frozenset({'2'}), frozenset({'4'}), frozenset({'1'}), frozenset({'3'})}


In [9]:
k=2

while True:

    # Ensure previous level of frequent itemsets exists for k-1
    if k - 1 not in frequent_itemsets or len(frequent_itemsets[k-1]) == 0:
        break

    # k-itemset candidates are generated from the (k-1)-itemsets
    candidates_k = generate_candidates(frequent_itemsets[k-1], frequent_1_itemsets, k)
    if not candidates_k:
        break

    # Candidates with infrequent subsets are removed
    candidates_k = prune_candidates(candidates_k, frequent_itemsets[k-1])
    if not candidates_k:
        break

    # Generate all k-itemsets (combinations of size k) from each transaction as frozensets, then pair each with 1 for counting
    transaction_k_itemsets = transactions.flatMap(lambda transaction: [frozenset(combo) for combo in combinations(transaction, k)]).map(lambda x: (x, 1))
    # Aggregate counts for each k-itemset by summing up their occurrences in the dataset
    transaction_k_counts = transaction_k_itemsets.reduceByKey(lambda a, b: a + b)
    # Convert candidate k-itemsets into an RDD, join it with the transaction counts, and extract the counts for valid candidates
    candidate_counts = sc.parallelize(list(candidates_k)).map(lambda item: (item, 1)).join(transaction_k_counts).map(lambda x: (x[0], x[1][1]))
    # Retain only candidate k-itemsets whose counts meet or exceed the frequency threshold and collect them into a list
    frequent_k_itemsets = candidate_counts.filter(lambda x: x[1] >= frequency_threshold).collect()

    if not frequent_k_itemsets:
        break

    frequent_itemsets[k] = set()
    for itemset, count in frequent_k_itemsets:
        frequent_itemsets[k].add(itemset)
        s_count[itemset] = count

    print(f"k={k}: {frequent_itemsets[k]}")

    k += 1

Previous freq itemsets: {frozenset({'5'}), frozenset({'2'}), frozenset({'4'}), frozenset({'1'}), frozenset({'3'})}

1 freq itemsets: {frozenset({'5'}), frozenset({'2'}), frozenset({'4'}), frozenset({'1'}), frozenset({'3'})}

Generate candidates for k=2
{frozenset({'3', '1'}), frozenset({'3', '4'}), frozenset({'3', '2'}), frozenset({'1', '4'}), frozenset({'1', '5'}), frozenset({'3', '5'}), frozenset({'2', '4'}), frozenset({'5', '4'}), frozenset({'5', '2'}), frozenset({'1', '2'})}

Pruned candidates: {frozenset({'3', '1'}), frozenset({'5', '2'}), frozenset({'1', '4'}), frozenset({'1', '5'}), frozenset({'3', '5'}), frozenset({'2', '4'}), frozenset({'5', '4'}), frozenset({'3', '2'}), frozenset({'3', '4'}), frozenset({'1', '2'})}
k=2: {frozenset({'1', '4'}), frozenset({'3', '2'}), frozenset({'1', '5'}), frozenset({'2', '4'}), frozenset({'5', '4'}), frozenset({'3', '4'}), frozenset({'1', '2'})}
Previous freq itemsets: {frozenset({'1', '4'}), frozenset({'3', '2'}), frozenset({'1', '5'}), froz

In [10]:
all_frequent_itemsets = []
for size, itemsets in frequent_itemsets.items():
    for itemset in itemsets:
        all_frequent_itemsets.append((frozenset(itemset), size, s_count[itemset]))

all_frequent_itemsets = sorted(all_frequent_itemsets, key=lambda x: (x[1], sorted(x[0])))

end_time_frequent_itemsets = time.time()

print("\nAll Frequent Itemsets:")
for itemset, size, count in all_frequent_itemsets:
    print(f"Itemset: {set(itemset)}, Size: {size}, Support Count: {count}")



All Frequent Itemsets:
Itemset: {'1'}, Size: 1, Support Count: 10
Itemset: {'2'}, Size: 1, Support Count: 8
Itemset: {'3'}, Size: 1, Support Count: 4
Itemset: {'4'}, Size: 1, Support Count: 5
Itemset: {'5'}, Size: 1, Support Count: 5
Itemset: {'1', '2'}, Size: 2, Support Count: 5
Itemset: {'1', '4'}, Size: 2, Support Count: 3
Itemset: {'1', '5'}, Size: 2, Support Count: 3
Itemset: {'3', '2'}, Size: 2, Support Count: 3
Itemset: {'2', '4'}, Size: 2, Support Count: 3
Itemset: {'3', '4'}, Size: 2, Support Count: 3
Itemset: {'5', '4'}, Size: 2, Support Count: 3
Itemset: {'3', '2', '4'}, Size: 3, Support Count: 3


In [11]:
execution_time_frequent_itemsets = end_time_frequent_itemsets - start_time_frequent_itemsets
print(f"Time to find frequent itemsets: {execution_time_frequent_itemsets:.4f} seconds")

Time to find frequent itemsets: 10.1350 seconds


## Association Rules

Association rules are relationships between items in a dataset, where the presence of certain items (antecedents) suggests the likelihood of the presence of other items (consequents). This is how association rules are generated:

1) Find frequent itemsets

2) For each frequent itemset, find its subsets (antecedents)

3) For each subset, create a rule where the subset is the antecedent, and the remaining items are the consequent

4) Calculate the confidence of the rule and keep only those rules that meet the confidence threshold (where confidence (A --> B) = Support(A U B) / Support(A))

In [12]:
start_time_association_rules = time.time()

association_rules = []

# Loop through all frequent itemsets
for itemset, size, count in all_frequent_itemsets:
    if size < 2: # Rules require at least one antecedent and one consequent
        continue

    # Generate all non-empty proper subsets (antecedents) of the itemset
    subsets = []
    for r in range(1, size):
        subsets.extend(combinations(itemset, r)) # Generate combinations of size r

    # Process each subset to form rules
    for subset in subsets:
        antecedent = frozenset(subset)
        consequent = itemset - antecedent # Calculate the consequent by subtracting antecedent from the itemset

        if not consequent: # Ensure the consequent is not empty
            continue

        # Retrieve support counts
        support_itemset = int(count) # Support count of the full itemset
        support_antecedent = int(s_count.get(antecedent, 0)) # Support count of the antecedent

        if support_antecedent == 0:
            continue  # Avoid division by zero

        # Calculate confidence: P(Consequent | Antecedent) = Support(Itemset) / Support(Antecedent)
        confidence = support_itemset / support_antecedent

        if confidence >= c:
            # Append the rule with its metrics to the list
            association_rules.append((set(antecedent), set(consequent), support_itemset, support_antecedent, confidence))


In [13]:
# Sort association rules by confidence in descending order
association_rules = sorted(association_rules, key=lambda x: x[4], reverse=True)

end_time_association_rules = time.time()

for antecedent, consequent, support_S, support_A, confidence in association_rules:
    print(f"Rule: {antecedent} -> {consequent}, Support(A U B): {support_S}, Support(A): {support_A}, Confidence: {confidence:.2f}")


Rule: {'3', '2'} -> {'4'}, Support(A U B): 3, Support(A): 3, Confidence: 1.00
Rule: {'3', '4'} -> {'2'}, Support(A U B): 3, Support(A): 3, Confidence: 1.00
Rule: {'2', '4'} -> {'3'}, Support(A U B): 3, Support(A): 3, Confidence: 1.00
Rule: {'3'} -> {'2'}, Support(A U B): 3, Support(A): 4, Confidence: 0.75
Rule: {'3'} -> {'4'}, Support(A U B): 3, Support(A): 4, Confidence: 0.75
Rule: {'3'} -> {'2', '4'}, Support(A U B): 3, Support(A): 4, Confidence: 0.75
Rule: {'2'} -> {'1'}, Support(A U B): 5, Support(A): 8, Confidence: 0.62
Rule: {'4'} -> {'1'}, Support(A U B): 3, Support(A): 5, Confidence: 0.60
Rule: {'5'} -> {'1'}, Support(A U B): 3, Support(A): 5, Confidence: 0.60
Rule: {'4'} -> {'2'}, Support(A U B): 3, Support(A): 5, Confidence: 0.60
Rule: {'4'} -> {'3'}, Support(A U B): 3, Support(A): 5, Confidence: 0.60
Rule: {'5'} -> {'4'}, Support(A U B): 3, Support(A): 5, Confidence: 0.60
Rule: {'4'} -> {'5'}, Support(A U B): 3, Support(A): 5, Confidence: 0.60
Rule: {'4'} -> {'3', '2'}, Supp

In [14]:
execution_time_association_rules = end_time_association_rules - start_time_association_rules
print(f"Time to generate association rules: {execution_time_association_rules:.4f} seconds")

Time to generate association rules: 0.0104 seconds
